## Winter Analysis Data Challenge :OO
### Import Libraries

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
from sqlalchemy import text, create_engine
import numpy as np
import plotly.express as px
import psycopg2
import psycopg2.extras
import json
import os
import zipfile
import requests
import time
from shapely.geometry import shape

### Setting up Datasets

In [4]:
# Station Entries
stationEntries = pd.read_csv('TrainStationEntriesExits/train-station-entries-exits-data-may-2025.csv')
# Station Entrance
stationEntrances = pd.read_csv('TrainStationEntranceLocations/stationentrances2020_v4.csv')
# Opal Patronage
opal = pd.read_csv('OpalPatronage/Opal_Patronage_20200101.txt', sep='|')

In [5]:
# Process multiple Opal files
data_folder = "OpalPatronage"
file_list = [f for f in os.listdir(data_folder) if f.startswith("Opal_Patronage_") and f.endswith(".txt")]

print(f"Found {len(file_list)} Opal files to process")

Found 2023 Opal files to process


In [6]:
df_list = [] 
for file in file_list: 
    file_path = os.path.join(data_folder, file) 
    df = pd.read_csv(file_path, sep="|") 
    df_list.append(df) 

# Combine all Opal data
opal_combined = pd.concat(df_list, ignore_index=True)
print(f"Combined Opal dataset shape: {opal_combined.shape}")

Combined Opal dataset shape: (1372294, 6)


### Data Cleaning and Conversion

In [8]:
# Cleaning Station Entries data...
# Step 1: Convert MonthYear to a datetime period (or datetime)
stationEntries['MonthYear'] = pd.to_datetime(stationEntries['MonthYear'], format='%b-%y', errors='coerce')

# Step 2: Replace 'Less than 50' with an estimate (like 49) or NaN
stationEntries['Trip'] = stationEntries['Trip'].replace('Less than 50', '49')

# Step 3: Convert Trip to numeric
stationEntries['Trip'] = pd.to_numeric(stationEntries['Trip'], errors='coerce')
stationEntries.rename(columns={'MonthYear': 'Date'}, inplace=True)

In [9]:
# Cleaning Station Entrances
stationEntrances['Street_Type'] = stationEntrances['Street_Type'].replace('<Null>', np.nan)

srid = 4283
stationEntrances['geom'] = gpd.points_from_xy(stationEntrances.LONG, stationEntrances.LAT)  # creating the geometry column
stationEntrances = stationEntrances.drop(columns=['LAT', 'LONG'])  # removing the old latitude/longitude fields
stationEntrances['geom'] = stationEntrances['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [10]:
# Cleaning Opal Numeric
def clean_opal_numeric(x): 
    """Clean numeric fields that may contain '<50', '<100' etc."""
    if isinstance(x, str) and "<" in x: 
        return 0  # Convert to 0 for analysis
    elif x is None: 
        return None
    else:
        return int(x)

def clean_mode_name(x): 
    """Remove UNKNOWN mode entries"""
    if x != "UNKNOWN": 
        return str(x)
    else:
        return None

# Create cleaned opal dataset
opal_clean = opal_combined.copy()

# Standardize column names to lowercase
opal_clean.columns = opal_clean.columns.str.lower()

# Clean numeric columns
opal_clean["tap_ons"] = opal_combined["Tap_Ons"].apply(clean_opal_numeric)
opal_clean["tap_offs"] = opal_combined["Tap_Offs"].apply(clean_opal_numeric)

# Clean mode names and remove UNKNOWN entries
opal_clean["mode_name"] = opal_combined["mode_name"].apply(clean_mode_name)
opal_clean = opal_clean.dropna(subset=["mode_name"])

# Convert date column
opal_clean['trip_origin_date'] = pd.to_datetime(opal_clean['trip_origin_date'])

print(f"Final cleaned Opal dataset shape: {opal_clean.shape}")

Final cleaned Opal dataset shape: (1327188, 6)


### Database Connection

In [12]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="winter"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', connect_args={"options": "-csearch_path=assignment"}, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    conn.commit()
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result